In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from datetime import date
import re
from dateutil.relativedelta import relativedelta

In [2]:
aapl = yf.Ticker("aapl")
aapl.option_chain(date="2022-08-26").calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL220826C00070000,2022-08-19 19:03:41+00:00,70.0,101.80,101.55,101.70,-2.299995,-2.209410,8,85,2.898440,True,REGULAR,USD
1,AAPL220826C00075000,2022-08-19 19:50:36+00:00,75.0,96.75,94.60,98.55,-2.949997,-2.958874,65,128,2.492191,True,REGULAR,USD
2,AAPL220826C00080000,2022-08-17 18:16:34+00:00,80.0,95.55,91.55,91.70,0.000000,0.000000,26,62,2.492191,True,REGULAR,USD
3,AAPL220826C00085000,2022-08-19 17:13:26+00:00,85.0,87.60,86.55,86.70,-0.650002,-0.736546,8,42,2.308598,True,REGULAR,USD
4,AAPL220826C00090000,2022-08-19 17:13:26+00:00,90.0,82.60,81.55,81.70,0.349998,0.425530,5,20,2.132817,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,AAPL220826C00220000,2022-08-10 13:40:55+00:00,220.0,0.01,0.00,0.01,0.000000,0.000000,2,80,0.625004,False,REGULAR,USD
59,AAPL220826C00225000,2022-08-02 13:31:59+00:00,225.0,0.01,0.00,0.01,0.000000,0.000000,1,21,0.687503,False,REGULAR,USD
60,AAPL220826C00235000,2022-08-11 15:37:19+00:00,235.0,0.01,0.00,0.01,0.000000,0.000000,1,165,0.781252,False,REGULAR,USD
61,AAPL220826C00245000,2022-07-29 17:38:58+00:00,245.0,0.01,0.00,0.01,0.000000,0.000000,1,1,0.875001,False,REGULAR,USD


In [9]:
stock_data = yf.download("aapl", (date.today() - relativedelta(days=30)).strftime("%Y-%m-%d"), date.today().strftime("%Y-%m-%d"), adjusted=True)
stock_returns = stock_data['Adj Close'].pct_change().dropna()
# stock_returns

[*********************100%***********************]  1 of 1 completed


In [16]:
def symbol_parts(symbol):
    r = re.search("^[A-Z]*", symbol)
    index = r.span()[1]
    ticker = symbol[0:index]
    year = 2000+ int(symbol[index:index + 2])
    month = int(symbol[index+2: index+4])
    day = int(symbol[index+4: index+6])
    opt_type = symbol[index+6: index+7]
    strike_price = int(symbol[index+7:]) / 1000
    return ticker, date(year, month, day), opt_type, strike_price


In [17]:
def get_sigma(stock_data):
    log_returns = np.array([stock_data[i+1]/stock_data[i] for i in range(len(stock_data)-1)])
    return np.sqrt(np.mean(log_returns**2))

def get_d1(sigma, t, s, x, r):
    return 1/(sigma * np.sqrt(t)) * (np.log(s/x) + (r + sigma**2/2)*t)

In [41]:
def option_price(symbol, r):
    ticker,t,option_type, x = symbol_parts(symbol)
    t = (t-date.today()).days/365
    stock_data = yf.download(ticker, (date.today() - relativedelta(days=30)).strftime("%Y-%m-%d"), date.today().strftime("%Y-%m-%d"), adjusted=True)['Adj Close']
    s = stock_data[-1]
    sigma = get_sigma(stock_data)
    d1 = get_d1(sigma, t, s,x,r)
    d2 = d1 - sigma * np.sqrt(t)
    return norm.cdf(d1)*s - norm.cdf(d2)*x*np.exp(-1*r*t)

option_price("AAPL220819C00030000", .091)

[*********************100%***********************]  1 of 1 completed


138.54231599798078

In [44]:
aapl = yf.Ticker("aapl")
calls = aapl.option_chain(date="2022-08-19").calls

priced_options = pd.DataFrame(columns=["contractSymbol", "price", "ask"])
for i in range(calls.shape[0]):
    price = option_price(calls.iloc[i].contractSymbol, .091)
    if price > calls.iloc[i].ask:
        priced_options.loc[i] = [calls.iloc[i].contractSymbol, price, calls.iloc[i].ask]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [54]:
pd.to_pickle(priced_options, "priced_options.pickle")